# Llm Classification Finetuning EDA


In [ ]:
import fastkaggle
import os
import shutil
import polars as pl
import torch
import numpy as np
from pathlib import Path

In [ ]:
api = fastkaggle.import_kaggle()

In [ ]:
data_base_path = Path("./data")
comp_name = "llm-classification-finetuning"
install_path = fastkaggle.setup_comp(comp_name)
datapath = data_base_path / comp_name
if os.path.exists(install_path):
    shutil.move(install_path, data_base_path / comp_name)

llm-classification-finetuning.zip: Skipping, found more recently modified local copy (use --force to force download)


# Dataset Description

The competition dataset consists of user interactions from the ChatBot Arena. In each user interaction a judge provides one or more prompts to two different large language models, and then indicates which of the models gave the more satisfactory response. The goal of the competition is to predict the preferences of the judges and determine the likelihood that a given prompt/response pair is selected as the winner.

Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. There are 55K rows in the training data, and you can expect roughly 25,000 rows in the test set.

## Files

### train.csv

- **id** - A unique identifier for the row.
- **model\_[a/b]** - The identity of model\_[a/b]. Included in train.csv but not test.csv.
- **prompt** - The prompt that was given as an input (to both models).
- **response\_[a/b]** - The response from model\_[a/b] to the given prompt.
- **winner*model*[a/b/tie]** - Binary columns marking the judge's selection. The ground truth target column.

### test.csv

- **id**
- **prompt**
- **response\_[a/b]**

### sample_submission.csv

A submission file in the correct format.

- **id**
- **winner*model*[a/b/tie]** - This is what is predicted from the test set.


In [ ]:
df_sample = pl.read_csv(datapath / "sample_submission.csv")
df_sample.head()

id,winner_model_a,winner_model_b,winner_tie
i64,f64,f64,f64
136060,0.333333,0.333333,0.333333
211333,0.333333,0.333333,0.333333
1233961,0.333333,0.333333,0.333333


In [ ]:
pl.Config.set_tbl_width_chars(300)
pl.Config.set_fmt_str_lengths(300)

polars.config.Config

In [ ]:
df_train = pl.read_csv(datapath / "train.csv")
df_test = pl.read_csv(datapath / "test.csv")
df_train.head()

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""[""Is it morally right to try to have a certain percentage of females on managerial positions?"",""OK, does pineapple belong on a pizza? Relax and give me fun answer.""]""","""[""The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Argume…","""[""As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better deci…",1,0,0
53567,"""koala-13b""","""gpt-4-0613""","""[""What is the difference between marriage license and marriage certificate?"",""How can I get both of them as quick as possible in California "",""What is the minimal time to get them? 1 day or 10 days?""]""","""[""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage…","""[""A marriage license and a marriage certificate are two different legal documents that have separate purposes.\n\n1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually obtained from local government or court officials before the weddi…",0,1,0
65089,"""gpt-3.5-turbo-0613""","""mistral-medium""","""[""explain function calling. how would you call a function?""]""","""[""Function calling is the process of invoking or executing a function in a programming language. To call a function, you need to use the function's name followed by parentheses \""()\"". Within the parentheses, you can pass arguments if the function requires any.\n\nHere is an example of how to call a…","""[""Function calling is the process of invoking a function in a program. A function is a block of code that performs a specific task and can be reused multiple times in a program. Functions help to organize and modularize code, making it more readable and maintainable.\n\nTo call a function, you simpl…",0,0,1
96401,"""llama-2-13b-chat""","""mistral-7b-instruct""","""[""How can I create a test set for a very rare category? I want to build a classifier.""]""","""[""Creating a test set for a very rare category can be challenging, as it may not be possible to collect a sufficient number of examples for the rare category. However, there are several strategies you can use to create a test set for a rare category:\n\n1. Use synthetic data: If the rare category is…","""[""When building a classifier for a very rare category, creating a test set can be challenging due to the limited amount of data available. Here are some steps you can take to create a test set for a rare category:\n\n1. Gather as much data as possible: The more data you have, the easier it will be t…",1,0,0
198779,"""koala-13b""","""gpt-3.5-turbo-0314""","""[""What is the best way to travel from Tel-Aviv to Jerusalem? Car? Bus? Plane?""]""","""[""The best way to travel from Tel Aviv to Jerusalem depends on your personal preference and the availability of transportation options. All the options you have mentioned are valid options, but here are some details to help you make your decision:\n\n* By car: Traveling by car is the quickest way …","""[""The best way to travel from Tel-Aviv to Jerusalem depends on your personal preferences and circumstances. \n\nIf you have a car and enjoy driving, then driving from Tel-Aviv to Jerusalem is an optio

In [ ]:
df_train[0, "prompt"]

'["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]'

In [ ]:
df_train[0, "response_a"]

'["The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\\n\\nHere are some arguments in favor of and against such policies:\\n\\n**Arguments in favor:**\\n\\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\\n\\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\\n\\n3. **Equality of Opportunity:** Setting targets for female representation in management can help ensure that women have equal opportunities to advance in their careers.\\n\\n4. **R